<a href="https://colab.research.google.com/github/sonenhur/k5_pytorch/blob/main/CatAndDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

In [ ]:
# 상수 정의
BATCH_SIZE = 128
NUM_WORKERS = 4

In [ ]:
# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize([256, 256]),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

In [ ]:
# 데이터셋 로드
train_dataset = ImageFolder(root='/data/catanddog/train', transform=transform)
test_dataset = ImageFolder(root='/data/catanddog/test', transform=transform)

In [ ]:
# 데이터 로더 생성
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [ ]:
# DataLoader 딕셔너리 생성
dataloaders = {'train': train_loader, 'val': test_loader}

------------------------------------------------------------------------------------------




## 모델 만들기
PyTorch에서 신경망 모델은 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) 을
상속받는 클래스(class)를 생성하여 정의합니다. ``__init__`` 함수에서 신경망의 계층(layer)들을 정의하고 ``forward`` 함수에서
신경망에 데이터를 어떻게 전달할지 지정합니다. 가능한 경우 GPU 또는 MPS로 신경망을 이동시켜 연산을 가속(accelerate)합니다.



In [ ]:
# 학습에 사용할 CPU나 GPU 장치
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
# LeNet 모델 정의
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5, padding='same'),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5, padding='same'),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 120, 7),
            nn.BatchNorm2d(120),
            nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Flatten(),
            nn.Linear(120*50*50, 64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(64, 2),
        )

    def forward(self, x):
        features = self.features(x)
        out = self.classifier(features)
        return out

model = LeNet().to(device)
print(model)

LeNet(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 120, kernel_size=(7, 7), stride=(1, 1))
    (9): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=300000, out_features=64, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=64, out_features=2, 

In [ ]:
# LeNet 모델의 예시 사용
input = torch.randn(BATCH_SIZE, 3, 224, 224).to(device)
output = model(input)
print(type(output), len(output), output.shape)

<class 'torch.Tensor'> 128 torch.Size([128, 2])


## ResNet 신경망 사용

In [ ]:
from torchvision import models
from torchvision.models import resnet50, ResNet50_Weights

# 미리 학습된 ResNet 모델 로드
model_ft = models.resnet50(weights='IMAGENET1K_V1')

# 매개변수 동결
def set_parameter(model, grad=False): #함수 생성하기 / 함수를 변수처럼 쓰기
  for param in model.parameters():
    param.requires_grad = False

set_parameter(model_ft, False)

# 분류기 조정
classifier = nn.Sequential(
    nn.Linear(2048, 256),  # ResNet-18 출력 크기와 일치하도록 입력 크기 조정
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(256, 64),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)
)

# ResNet의 완전연결 레이어를 사용자 커스텀 분류기로 교체
model_ft.fc = classifier
model = model_ft.to(device)

In [ ]:
from torchinfo import summary

def summary_model(model):
  print(summary(model=model, input_size=(32, 3, 224, 224), col_width=20,
        col_names=['input_size', 'output_size', 'num_params', 'trainable'], row_settings=['var_names'], verbose=0))


summary_model(model)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 2]              --                   Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    └─Bottleneck (0)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     (4,096)              False
│    │    

------------------------------------------------------------------------------------------




## 모델 매개변수 최적화하기
모델을 학습하려면 [손실 함수(loss function)](https://pytorch.org/docs/stable/nn.html#loss-functions) 와
[옵티마이저(optimizer)](https://pytorch.org/docs/stable/optim.html) 가 필요합니다.



In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

각 학습 단계(training loop)에서 모델은 (배치(batch)로 제공되는) 학습 데이터셋에 대한 예측을 수행하고,
예측 오류를 역전파하여 모델의 매개변수를 조정합니다.



In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    total_loss = 0.0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch % 100 == 0:
            current = batch * len(X)
            print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")
    return total_loss / len(dataloader)

모델이 학습하고 있는지를 확인하기 위해 테스트 데이터셋으로 모델의 성능을 확인합니다.



In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습 단계는 여러번의 반복 단계 (*에폭(epochs)*) 를 거쳐서 수행됩니다. 각 에폭에서는 모델은 더 나은 예측을 하기 위해  매개변수를 학습합니다.
각 에폭마다 모델의 정확도(accuracy)와 손실(loss)을 출력합니다; 에폭마다 정확도가 증가하고 손실이 감소하는 것을 보려고 합니다.



In [ ]:
# 학습
epochs = 4
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(train_loader, model, loss_fn, optimizer)
    print(f"Training Loss: {train_loss}")
    test(test_loader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 0.696088  [    0/  385]
Training Loss: 0.6718061119318008
Test Error: 
 Accuracy: 50.0%, Avg loss: 0.683744 

Epoch 2
-------------------------------
loss: 0.684438  [    0/  385]
Training Loss: 0.6991800218820572
Test Error: 
 Accuracy: 50.0%, Avg loss: 0.675025 

Epoch 3
-------------------------------
loss: 0.679315  [    0/  385]
Training Loss: 0.6627065390348434
Test Error: 
 Accuracy: 63.3%, Avg loss: 0.662511 

Epoch 4
-------------------------------
loss: 0.669964  [    0/  385]
Training Loss: 0.6507727056741714
Test Error: 
 Accuracy: 72.4%, Avg loss: 0.653045 

Done!


[모델을 학습하는 방법](optimization_tutorial.html) 을 자세히 알아보세요.




argmax  
0.7 / 0.3 에서 0 나온다 -> cat  
0.3 / 0.7 에서 1 나온다 -> dog  

------------------------------------------------------------------------------------------




## 모델 저장하기
모델을 저장하는 일반적인 방법은 (모델의 매개변수들을 포함하여) 내부 상태 사전(internal state dictionary)을
직렬화(serialize)하는 것입니다.



In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 모델 불러오기

모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함됩니다.



In [ ]:
model = LeNet().to(device)
model.load_state_dict(torch.load("model.pth"))

RuntimeError: Error(s) in loading state_dict for LeNet:
	Missing key(s) in state_dict: "features.0.weight", "features.0.bias", "features.1.weight", "features.1.bias", "features.1.running_mean", "features.1.running_var", "features.4.weight", "features.4.bias", "features.5.weight", "features.5.bias", "features.5.running_mean", "features.5.running_var", "features.8.weight", "features.8.bias", "features.9.weight", "features.9.bias", "features.9.running_mean", "features.9.running_var", "classifier.2.weight", "classifier.2.bias", "classifier.5.weight", "classifier.5.bias". 
	Unexpected key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "bn1.num_batches_tracked", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.bn1.num_batches_tracked", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_v

이제 이 모델을 사용해서 예측을 할 수 있습니다.



In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

[모델을 저장하고 불러오는 방법](saveloadrun_tutorial.html) 을 자세히 알아보세요.


